In [ ]:
# Pick directory --> load all data into study dict

# study --> A1, A2, ... --> A1S1, A1S2, ... A2S1, A2S2, ... --> A1S1C1, A1S1C2, ..., A1S2C1, ...

# sort cell data, match across waveforms

# batch run spikeAnalysisGUI (firing data) {batch_ratemaps.py}

# batch run spikeWidthGUI (spike width)

# batch run PowerSpectrum.vel --> save output LFP data and load into python

# batch run hfoGUI (LFP vizualization) 

# batch run PRISM/neurofunc (field properties) {batch_ratemaps.py}

# batch run odor/object analysis